In [128]:
import numpy as np
from sklearn.preprocessing import LabelEncoder
import pandas as pd

In [129]:
# importing the dataset
df = pd.read_csv("iris.csv")
df = df.drop(["Id"],axis=1)
df['Species']=LabelEncoder().fit_transform(df['Species'])

,SepalLengthCm,SepalWidthCm,PetalLengthCm,PetalWidthCm,Species
0,5.1,3.5,1.4,0.2,0
1,4.9,3.0,1.4,0.2,0
2,4.7,3.2,1.3,0.2,0
3,4.6,3.1,1.5,0.2,0
4,5.0,3.6,1.4,0.2,0
...,...,...,...,...,...
145,6.7,3.0,5.2,2.3,2
146,6.3,2.5,5.0,1.9,2
147,6.5,3.0,5.2,2.0,2
148,6.2,3.4,5.4,2.3,2


In [155]:
df_train=pd.DataFrame()
df_test=pd.DataFrame()

In [158]:
for i in df['Species'].unique():
    df_train = pd.concat([df_train, df[df['Species'] == i].iloc[:40]],ignore_index=True)
    df_test = pd.concat([df_test, df[df['Species'] == i].iloc[40:]],ignore_index=True)
    
df_train

,SepalLengthCm,SepalWidthCm,PetalLengthCm,PetalWidthCm,Species
0,5.1,3.5,1.4,0.2,0
1,4.9,3.0,1.4,0.2,0
2,4.7,3.2,1.3,0.2,0
3,4.6,3.1,1.5,0.2,0
4,5.0,3.6,1.4,0.2,0
...,...,...,...,...,...
115,7.7,3.0,6.1,2.3,2
116,6.3,3.4,5.6,2.4,2
117,6.4,3.1,5.5,1.8,2
118,6.0,3.0,4.8,1.8,2


In [159]:
y_train=df_train["Species"]
X_train=df_train.drop(["Species"],axis=1)
y_test=df_test["Species"]
X_test=df_test.drop(["Species"],axis=1)

print(X_test)

    SepalLengthCm  SepalWidthCm  PetalLengthCm  PetalWidthCm
0             5.0           3.5            1.3           0.3
1             4.5           2.3            1.3           0.3
2             4.4           3.2            1.3           0.2
3             5.0           3.5            1.6           0.6
4             5.1           3.8            1.9           0.4
5             4.8           3.0            1.4           0.3
6             5.1           3.8            1.6           0.2
7             4.6           3.2            1.4           0.2
8             5.3           3.7            1.5           0.2
9             5.0           3.3            1.4           0.2
10            5.5           2.6            4.4           1.2
11            6.1           3.0            4.6           1.4
12            5.8           2.6            4.0           1.2
13            5.0           2.3            3.3           1.0
14            5.6           2.7            4.2           1.3
15            5.7       

In [160]:
def corr_coef(x, y):
    mean_x = x.mean()
    mean_y = y.mean()
    numerator = np.sum((x - mean_x) * (y - mean_y))
    denominator = np.sqrt(np.sum((x - mean_x) ** 2) * np.sum((y - mean_y) ** 2))
    corr = numerator / denominator
    return corr

In [161]:
def cov_matrix(df):
    corr_matrix = pd.DataFrame(index=df.columns, columns=df.columns)
    for col1 in df.columns:
        for col2 in df.columns:
            corr_matrix.loc[col1, col2] = corr_coef(df[col1], df[col2])

    corr_matrix = corr_matrix.astype(float)
    # Calculate the standard deviations of the variables
    std_devs = np.std(df, axis=0)

    # Calculate the covariance matrix
    covariance_matrix = corr_matrix * np.outer(std_devs, std_devs)
    
    return covariance_matrix

In [164]:
def multivariate_normal_pdf(x, mean, covariance):
    n = len(x)
    det_covariance = np.linalg.det(covariance)
    inv_covariance = np.linalg.inv(covariance)
    x_minus_mean = x - mean
    exponent = -0.5 * np.dot(np.dot(x_minus_mean, inv_covariance), x_minus_mean)
    coefficient = 1 / ((2 * np.pi) ** (n / 2) * np.sqrt(det_covariance))
    return coefficient * np.exp(exponent)

def bayes_classifier(x):
    class_probs = []
    for i in range(len(np.unique(y_train))):
        class_mean = class_means[i]
        class_covariance = class_covs[i]
        class_probs.append(multivariate_normal_pdf(x, class_mean, class_covariance))
    return np.argmax(class_probs)

# Calculate the mean and covariance matrix for each class in the training set
class_means = []
class_covs = []
for class_label in np.unique(y_train):
    X_class = X_train[y_train == class_label]
    class_means.append(np.mean(X_class, axis=0))
    class_covs.append(cov_matrix(X_class))

# Classify the test feature vectors
y_pred = np.array([bayes_classifier(X_test.iloc[x].values) for x in range(len(X_test))])

# Calculate the accuracy of classification
accuracy = np.mean(y_pred == y_test)
print("Accuracy:", accuracy)


Accuracy: 1.0


In [165]:
comparing = pd.DataFrame({'Actual': y_test, 'Predicted': y_pred})

print(comparing)

    Actual  Predicted
0        0          0
1        0          0
2        0          0
3        0          0
4        0          0
5        0          0
6        0          0
7        0          0
8        0          0
9        0          0
10       1          1
11       1          1
12       1          1
13       1          1
14       1          1
15       1          1
16       1          1
17       1          1
18       1          1
19       1          1
20       2          2
21       2          2
22       2          2
23       2          2
24       2          2
25       2          2
26       2          2
27       2          2
28       2          2
29       2          2
